### Problem Statement: You have to perform the following operation using LLM with the help of function calling

Eg. A+7=H, B+2=D, C-1=B.

Using Langraph make an agentic model which will take query from user (i.e. add/subtract 5 steps to G)


In [ ]:
# Import the necessary libraries
import os
from typing import Literal
from langchain_core.messages import HumanMessage
from langchain_groq import ChatGroq
from langchain_core.tools import tool
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import END, StateGraph, MessagesState
from langgraph.prebuilt import ToolNode

In [ ]:
# Define the groq cloud api key
os.environ["GROQ_API_KEY"] = "your_groq_cloud_api_key"

Defining the perform arithmetic function as a langgraph tool


In [13]:
# Define a new graph
# Define the function as the tool node which will be the entry point to perform arithmetic operations
@tool("some_function")
def perform_letter_arithmetic(expression: str) -> str:
    """Perform arithmetic operations on letters"""
    expression = expression.strip().lower()
    
    if 'add' in expression or 'subtract' in expression:
        parts = expression.split(' to ')
        if len(parts) == 2:
            operation_part, letters_part = parts
            operation = 'add' if 'add' in operation_part else 'subtract'
            
            # Extract the number and letters from the string
            operation_part = operation_part.replace('add', '').replace('subtract', '')
            steps = list(map(int, operation_part.split(', ')))
            letters = letters_part.replace('respectively', '').strip().split(', ')
            
            if len(steps) != len(letters):
                raise ValueError("Number of steps and letters not equal in query")
            
            results = []
            for letter, step in zip(letters, steps):
                ascii_value = ord(letter.upper())
                if operation == 'add':
                    new_ascii_value = ascii_value + step
                elif operation == 'subtract':
                    new_ascii_value = ascii_value - step
                # Wrap around to 'A' if new_ascii_value is less than 'A'
                if new_ascii_value < ord('A'):
                    new_ascii_value += 26
                # Wrap around to 'Z' if new_ascii_value is greater than 'Z'
                elif new_ascii_value > ord('Z'):
                    new_ascii_value -= 26
                results.append(chr(new_ascii_value))
            return ", ".join(results)
    
    else:
        operation, letter, steps = None, None, None
        expression = expression.strip()
        if '+' in expression:
            operation = 'add'
            letter, steps = expression.split('+')
        elif '-' in expression:
            operation = 'subtract'
            letter, steps = expression.split('-')
        else:
            raise ValueError("Invalid expression. Please use '+' or '-' to separate the character")
        
        steps = int(steps)
        ascii_value = ord(letter.upper())
        if operation.lower() == 'add':
            new_ascii_value = ascii_value + steps
        elif operation.lower() == 'subtract':
            new_ascii_value = ascii_value - steps
        else:
            raise ValueError("Invalid operation. Please use 'add' or 'subtract'.")
        
        # Wrap around to 'A' if new_ascii_value is less than 'A'
        if new_ascii_value < ord('A'):
            new_ascii_value += 26
        # Wrap around to 'Z' if new_ascii_value is greater than 'Z'
        elif new_ascii_value > ord('Z'):
            new_ascii_value -= 26
        
        return chr(new_ascii_value)

Adding the function to the ToolNode Class


In [ ]:
tools = [perform_letter_arithmetic]

tool_node = ToolNode(tools)

# Define the groq model using the ChatGroq class
model = ChatGroq(model_name="llama3-8b-8192", temperature=0).bind_tools(tools)

Adding the boilerplate to perform the workflow of the graph from agent's entry_mode to other nodes till END


In [ ]:
# Define the functions to continue or end and the entry point for call_model, SOURCE: https://langchain-ai.github.io/langgraph/#example
def should_continue(state: MessagesState) -> Literal["tools", END]:
    """
    This function is called after each message is processed. It determines whether the conversation should continue or end
    using the continuous edges of the langgraph API.
    """
    messages = state['messages']
    last_message = messages[-1]
    # If the LLM makes a tool call, then we route to the "tools" node
    if last_message.tool_calls:
        return "tools"
    # Otherwise, we stop (reply to the user)
    return END

def call_model(state: MessagesState):
    """Takes the input state as MessagesState and returns the output of the required response using model invoke"""
    messages = state['messages']
    response = model.invoke(messages)
    # We return a list, because this will get added to the existing list
    return {"messages": [response]}

workflow = StateGraph(MessagesState)

# Define the two nodes we will cycle between
workflow.add_node("agent", call_model)
workflow.add_node("tools", tool_node)

# Set the entrypoint as `agent` which is the first point in graph to be called 
workflow.set_entry_point("agent")

# Add a conditional edge which first calls the agent as the start node and next we pass in function to define what node is called next
workflow.add_conditional_edges(
    "agent",
    should_continue,
)

# Add a normal edge from `tools` to `agent`, after tool agent is called
workflow.add_edge("tools", 'agent')

# Initialize memory to persist state between graph runs
checkpointer = MemorySaver()

# Compile the graph
app = workflow.compile(checkpointer=checkpointer)

Performing the query based on different operations


In [ ]:
# The query to give to the model
queries = ['A+3','B-2','N+1','P+3']

for query in queries:
# Recieve the output from the model
    final_state = app.invoke(
                {"messages": [HumanMessage(content=query)]},
                config={"configurable": {"thread_id": id(query)}}  # Use a unique thread_id for each query
            )

    print()
    print(query+' = '+final_state["messages"][-1].content)

In [ ]:
# Performing the query without using AI agent
queries = ['A+3','B-2','N+1','P+3', 'add 4, 2, 7 to A, B, G respectively']
for query in queries:
    print()
    print(query+' = '+perform_letter_arithmetic(query))